<a href="https://colab.research.google.com/github/DavidVollendroff/DS-Unit-2-Applied-Modeling/blob/master/module2/LS_DS_232_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 

In [0]:
import pandas as pd
import numpy as np

seasons = [yr for yr in range(1990, 2021)]
data = {}
stats_url = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'
for season in seasons:
  stats = pd.read_html(stats_url.format(season))
  centers = stats[0][stats[0]['Pos']=='C'].dropna(axis=1)
  data['{}-{}'.format(season -1, str(season)[-2:])] = centers

wins_dict = {}
standings_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"
for season in seasons:
  standings = pd.read_html(standings_url.format(season))
  standings = pd.concat(standings[:2])
  # wins = standings[0]['W'].dropna(axis=1)
  wins_dict['{}-{}'.format(season -1, str(season)[-2:])] = standings

for key in wins_dict.keys():
  wins_dict[key] = wins_dict[key][~wins_dict[key]['GB'].str.contains('Division')]
  wins_dict[key]['Team'] = pd.concat([wins_dict[key]['Eastern Conference'], wins_dict[key]['Western Conference']], axis=0).dropna()
  wins_dict[key] = wins_dict[key].drop(['Eastern Conference', 'Western Conference'], axis=1)

team_abr = ['OKC', 'MIA', 'MIN', 'SAS', 'BRK', 'IND', 'NOP', 'CHI',
       'BOS', 'GSW', 'ORL', 'HOU', 'LAL', 'UTA', 'SAC', 'PHO', 'POR',
       'MEM', 'ATL', 'DET', 'PHI', 'CLE', 'WAS', 'LAC', 'MIL', 'NYK',
       'CHO', 'DEN', 'DAL', 'TOR', 'CHA', 'NOH', 'NJN', 'SEA', 'NOK',
       'CHH', 'VAN', 'WSB']
team_cities = ['Oklahoma City', 'Miami', 'Minnesota', 'San Antonio', 'Brooklyn', 'Indiana',
            'New Orleans', 'Chicago', 'Boston', 'Golden State', 'Orlando', 'Houston',
            'Los Angeles Lakers', 'Utah', 'Sacramento', 'Phoenix', 'Portland', 'Memphis',
            'Atlanta', 'Detroit', 'Philadelphia', 'Cleveland', 'Washington', 'Los Angeles Clippers',
            'Milwaukee', 'New York', 'Charlotte', 'Denver', 'Dallas', 'Toronto', 'Charlotte',
            'New Orleans', 'New Jersey', 'Seattle', 'New Orleans', 'Charlotte', 'Vancouver',
            'Washington']

abr_city = dict(zip(team_abr, team_cities))

# All-NBA Awards List
url = 'https://www.basketball-reference.com/awards/all_league.html'
all_nba = pd.read_html(url) # pulls from the web
all_nba = all_nba[0] # removes from the list
all_nba = all_nba.dropna() # drops spacer rows
all_nba = all_nba[['Season', 'Unnamed: 3']] # selects only the Centers
all_nba = all_nba.set_index('Season') # removes useless index

def trim(some_string):
  return some_string[:-2]
all_nba = all_nba['Unnamed: 3'].apply(trim)

def hof_remover(some_string):
  if some_string.endswith('*'):
    return some_string[:-1]
  else:
    return some_string

for key in data.keys():
  data[key]['Player'] = data[key]['Player'].apply(hof_remover)

modern_all_nba = pd.DataFrame(all_nba[(all_nba.index >'1988-89')])


In [0]:
awards = np.array([1 for i in range(90)])
modern_all_nba['awards'] = awards
modern_all_nba.rename(columns={'Unnamed: 3': 'Player'}, inplace=True)

merged_data = {}
for key in modern_all_nba.index.unique():
  merged_data[key] = pd.merge(data[key], modern_all_nba[modern_all_nba.index==key], how='left', on='Player')
  merged_data[key].fillna(0, inplace=True)
  merged_data[key]['Season'] = np.repeat(key, len(merged_data[key]))

latest_seasons = ['2018-19', '2019-20']
df_list = []
for key in merged_data.keys():
  if key not in latest_seasons:
    df_list.append(merged_data[key])
train = pd.concat(df_list, axis=0)
validate = merged_data[latest_seasons[0]]
test = data[latest_seasons[1]]

In [0]:
target = 'awards'

features = train.columns.tolist()

removed_features = ['Player',
                    'Pos',
                    'Rk',
                    'awards',
                    'Season' #may want to keep
                    ]

for item in removed_features:
  if item in features:
    features.remove(item)


- [ ] For the evaluation metric you chose, what score would you get just by guessing?

In [9]:
baseline = validate[target].value_counts(normalize=True).max()*100 # dense code
print(f'Accuracy by guessing: {baseline}%')

Accuracy by guessing: 97.5%



- [X] Can you make a fast, first model that beats guessing?

Short Answer: Sort of

In [10]:
!pip install category-encoders

     |████████████████████████████████| 102kB 2.3MB/s 


In [11]:
# features matrix and target vector 
X_train = train[features]
y_train = train[target]
X_val = validate[features]
y_val = validate[target]
X_test = test[features]

# import all necessary functions for sklearn pipeline
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

# create pipeline
pipe = make_pipeline(ce.OrdinalEncoder(),
                         RandomForestClassifier(max_depth=None, n_estimators=1200))
pipe.fit(X_train, y_train);

pipe.score(X_val, y_val)

0.9583333333333334

In [17]:
validate[(pipe.predict(X_val)==True)]

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,awards,Season
22,126,Anthony Davis,C,25,NOP,56,1850,30.3,.597,.141,.422,9.9,27.5,18.8,19.0,2.2,6.0,8.4,29.5,6.4,3.1,9.5,.247,4.7,3.9,8.5,4.9,0.0,2018-19
106,492,Nikola Vučević,C,28,ORL,80,2510,25.5,.573,.171,.168,9.4,31.9,20.5,21.9,1.6,3.0,9.9,28.0,5.4,4.7,10.1,.193,3.0,3.4,6.4,5.3,0.0,2018-19


In [31]:
validate[validate.loc[:, 'Player'] == 'Joel Embiid']

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,awards,Season
30,155,Joel Embiid,C,24,PHI,64,2154,26.1,.593,.219,.541,8.1,34.0,21.4,18.4,1.0,4.4,13.2,33.3,4.9,3.8,8.7,.194,2.0,2.1,4.1,3.3,1.0,2018-19


In [0]:
new_validate = validate.copy()

In [0]:
new_validate['pred_prob'] = pipe.predict_proba(X_val)[:,1]

In [0]:
pd.options.display.max_rows = 500

In [79]:
new_validate.sort_values(by='pred_prob', ascending=False)

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,awards,Season,pred_prob
22,126,Anthony Davis,C,25,NOP,56,1850,30.3,.597,.141,.422,9.9,27.5,18.8,19.0,2.2,6.0,8.4,29.5,6.4,3.1,9.5,.247,4.7,3.9,8.5,4.9,0.0,2018-19,0.589167
106,492,Nikola Vučević,C,28,ORL,80,2510,25.5,.573,.171,.168,9.4,31.9,20.5,21.9,1.6,3.0,9.9,28.0,5.4,4.7,10.1,.193,3.0,3.4,6.4,5.3,0.0,2018-19,0.545000
29,147,Andre Drummond,C,25,DET,79,2647,23.4,.555,.036,.392,16.8,34.7,25.4,7.2,2.5,4.4,12.4,22.9,4.1,5.9,10.0,.181,-0.7,3.6,2.9,3.3,0.0,2018-19,0.500000
43,187,Rudy Gobert,C,26,UTA,81,2577,24.6,.682,.000,.733,13.2,30.2,21.9,9.6,1.2,5.8,12.1,17.8,8.7,5.7,14.4,.268,2.0,5.1,7.0,5.9,1.0,2018-19,0.438333
0,4,Steven Adams,C,25,OKC,80,2669,18.5,.591,.002,.361,14.7,14.8,14.7,6.6,2.0,2.4,12.6,16.4,5.1,4.0,9.1,.163,0.6,2.1,2.7,3.2,0.0,2018-19,0.315000
100,480,Karl-Anthony Towns,C,23,MIN,77,2545,26.3,.622,.270,.342,10.9,29.3,20.0,17.2,1.3,4.2,13.7,28.9,7.2,3.2,10.4,.197,4.8,2.0,6.8,5.7,0.0,2018-19,0.298333
101,485,Myles Turner,C,22,IND,74,2119,18.0,.567,.251,.258,5.4,22.0,13.9,8.0,1.4,8.4,10.3,20.0,1.9,4.4,6.3,.144,-1.4,4.7,3.3,2.8,0.0,2018-19,0.255000
69,310,Brook Lopez,C,30,MIL,81,2322,14.6,.597,.651,.169,1.5,15.0,8.7,5.7,1.0,6.5,8.9,16.7,2.3,4.3,6.7,.138,-0.2,2.9,2.7,2.8,0.0,2018-19,0.227500
45,208,Montrezl Harrell,C,25,LAC,82,2158,23.4,.636,.019,.466,9.3,16.7,13.1,12.5,1.6,4.0,11.0,23.5,6.1,2.6,8.7,.193,2.6,1.6,4.2,3.4,0.0,2018-19,0.215833
37,180,Marc Gasol,C,34,TOT,79,2436,16.7,.546,.314,.291,3.6,25.1,14.3,22.2,1.7,3.2,13.6,20.7,2.0,4.0,6.0,.118,-0.3,3.1,2.8,3.0,0.0,2018-19,0.188333


### Only two players are predicted to make the team, even though three are required by definition. 

So, if we were to say that we'll choose the top 3 players according to predicted probability we're choosing to define the correct answer as, "Which three players will make the All-NBA Teams?" That's a different question than, "Will a given player make the team?"

Guessing randomly which players will make the team has a nearly zero probability of being correct. Approximately 0.00000356074. But guessing whether an individual player will make the team is 97.5% accurate.

I think I can do better than the baseline, quite easily, at guessing which three players will make the team. But in order to beat the accuracy of guessing for individual players will necessitate that I guess at least 2 of the 3 players to make the team. 
  

# Additionally my currently collected data does a poor job of accounting for the way people actually vote. My hypothesis is that while Nikola Jokić didn't make the Top Ten in my model's predictions, he was rewarded for leading his team to the best record in the conference. And Anthony Davis only received a single vote for the 3rd team because he demanded a trade midway through the season. It was bad PR.